In [1]:
import pandas as pd
import numpy as np 
import geopandas as gpd
from bokeh.models.glyphs import Patches
from bokeh.plotting import figure, save,gmap
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, HoverTool,GMapOptions
import pyproj

import pandas as pd   
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Range1d,HoverTool
from bokeh.layouts import layout,column,row
from bokeh.palettes import Spectral3
#from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.tile_providers import STAMEN_TERRAIN
from pyproj import Proj, transform
output_file('mapping_targets.html')



In [2]:
Species=gpd.read_file("D:\\Qgis\\Species_IMP_MH\\Copy of Species_Imp_MH.shp")
Birds=gpd.read_file("D:\\BNHS_Project\\MM_B_ST\\MM_B_ST_Review.shp")
#Chirps=gpd.read_file("C:\\Users\\Sarvesh\\Downloads\\Dec2019SpChecklist.geojson")
Corals=gpd.read_file("C:/Users/User/Downloads/corals.geojson")
Mang=gpd.read_file("C:\\Users\\User\\Downloads\\mangupdat1.geojson")

In [3]:
#helper function to convert lat/long to easting/northing for mapping
#this relies on functions from the pyproj library
def LongLat_to_EN(long, lat):
    try:
      easting, northing = transform(
        Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
      return easting, northing
    except:
      return None, None 

In [4]:
Species['E'], Species['N'] = zip(*Species.apply(lambda x: LongLat_to_EN(x['Long'],x['Lat']), axis=1))
Birds['E'], Birds['N'] = zip(*Birds.apply(lambda x: LongLat_to_EN(x['Long'],x['Lat']), axis=1))


In [5]:
Corals_Web = Corals.to_crs({'init': 'epsg:3857'})

def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )
    
    

In [6]:
Mang_Web = Mang.to_crs({'init': 'epsg:3857'})

def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )
    
    

In [7]:
Corals_Web['x'] = Corals_Web.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
Corals_Web['y'] = Corals_Web.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)



In [8]:
Mang_Web['x'] = Mang_Web.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
Mang_Web['y'] = Mang_Web.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)



In [9]:
x=Corals_Web['x'].tolist() 
y=Corals_Web['y'].tolist()

In [10]:
x=Mang_Web['x'].tolist() 
y=Mang_Web['y'].tolist()

In [11]:
Species=Species.drop('geometry', axis=1).copy()
Species_Source = ColumnDataSource(Species)

Species_hover=HoverTool()
Species_hover.tooltips = [('line', '@Species')]



In [12]:
Birds=Birds.drop('geometry', axis=1).copy()
Birds_Source = ColumnDataSource(Birds)

Birds_hover=HoverTool()
Birds_hover.tooltips = [('Location', '@Location')]

In [13]:
Corals_bokeh= Corals_Web.drop('geometry', axis=1).copy()
Corals_Source = ColumnDataSource(Corals_bokeh)



In [14]:
Mang_bokeh=Mang_Web.drop('geometry', axis=1).copy()
Mang_Source = ColumnDataSource(Mang_bokeh)



In [15]:

Corals_hover=HoverTool()
Corals_hover.tooltips=[('Corals','@Name')]

In [16]:

Mang_hover=HoverTool()
Mang_hover.tooltips=[('Mangrove','@Name')]

In [17]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

#left = -2150000  
#right = 18000000
#bottom = -5300000
#top = 11000000
left =8300000
right = 8700000
bottom =2200000
top =2700000

plot1 = figure(x_range=Range1d(left, right), y_range=Range1d(bottom, top),tools="pan,wheel_zoom,box_zoom,reset",plot_width=1000, plot_height=800)
plot1.add_tile(STAMEN_TERRAIN)


TileRenderer(id='1038', ...)

In [18]:
plot1.circle(x='E', y='N', source=Species_Source, line_color='grey', fill_color='yellow')
plot1.add_tools(Species_hover)
plot1.axis.visible = False




In [19]:
#plot2= figure(plot_width=800, plot_height=500,tools="pan,wheel_zoom,box_zoom,reset")
plot2= figure(x_range=Range1d(left, right), y_range=Range1d(bottom, top),tools="pan,wheel_zoom,box_zoom,reset",plot_width=1000, plot_height=800)
plot2.add_tile(STAMEN_TERRAIN)
plot2.circle(x='E', y='N', source=Birds_Source, line_color='grey', fill_color='red')
plot2.add_tools(Birds_hover) 
plot2.axis.visible = False
#tab2 = Panel(child=plot2, title="Birds") 

In [20]:
#plot2= figure(plot_width=800, plot_height=500,tools="pan,wheel_zoom,box_zoom,reset")
plot3= figure(x_range=Range1d(left, right), y_range=Range1d(bottom, top),tools="pan,wheel_zoom,box_zoom,reset",plot_width=1000, plot_height=800)
plot3.add_tile(STAMEN_TERRAIN)
plot3.patches(xs='x', ys='y', source=Corals_Source,fill_color="#fb9a99",fill_alpha=10.3)
plot3.add_tools(Corals_hover) 
plot3.axis.visible = False
tab3 = Panel(child=plot3, title="Corals") 

In [21]:
#plot2= figure(plot_width=800, plot_height=500,tools="pan,wheel_zoom,box_zoom,reset")
plot4= figure(x_range=Range1d(left, right), y_range=Range1d(bottom, top),tools="pan,wheel_zoom,box_zoom,reset",plot_width=1000, plot_height=800)
plot4.add_tile(STAMEN_TERRAIN)
plot4.patches(xs='x', ys='y', source=Mang_Source,fill_color="blue",fill_alpha=10.3)
plot4.add_tools(Mang_hover) 
plot4.axis.visible = False
tab4 = Panel(child=plot4, title="Mangroves") 

In [22]:
#plot2= figure(plot_width=800, plot_height=500,tools="pan,wheel_zoom,box_zoom,reset")
plot5= figure(x_range=Range1d(left, right), y_range=Range1d(bottom, top),tools="pan,wheel_zoom,box_zoom,reset",plot_width=1000, plot_height=800)
plot5.add_tile(STAMEN_TERRAIN)
plot5.patches(xs='x', ys='y', source=Corals_Source,fill_color="#fb9a99",fill_alpha=10.3)
plot5.patches(xs='x', ys='y', source=Mang_Source,fill_color="blue",fill_alpha=10.3)
plot5.circle(x='E', y='N', source=Birds_Source, line_color='grey', fill_color='red')
plot5.circle(x='E', y='N', source=Species_Source, line_color='grey', fill_color='yellow')
#plot5.add_tools(Corals_hover) 
plot5.axis.visible = False
tab5 = Panel(child=plot5, title="All") 

In [23]:
#show(tabs)

from bokeh.palettes import Spectral5
from bokeh.transform import factor_cmap
#output_file('slider.html')
# Barplot showing no of Occorances of Species 
from bokeh.palettes import Spectral5


In [24]:
from bokeh.transform import factor_cmap
grouped = Species.groupby('Species').count()
source = ColumnDataSource(grouped)
Species_list= source.data['Species'].tolist()
bar = figure(x_range=Species_list)
bar_hover=HoverTool()
bar_hover.tooltips = [('No of Occourance', '@Type')]
bar.vbar(x='Species', top='Type', source=source,width=0.4)
bar.add_tools(bar_hover)
#show(bar) 

#from bokeh.layouts import gridplot
#output_file('Dashboard.html')
#dash = gridplot([[tabs,bar]])

# show the results

#show(dash)


In [25]:
Type_grouped=Species.groupby('Type').count()
Source_Type=ColumnDataSource(Type_grouped)
Type_list=Source_Type.data['Type'].tolist()
Y_axis= Type_grouped['Village'].tolist()
scatter=figure(x_range=Type_list,plot_width=400, plot_height=400)
scatter_hover=HoverTool()
scatter_hover.tooltips = [('No of Occourance','@Type')]
scatter.circle(x=Type_list,y=Y_axis,size=25,fill_color="orange")
scatter.add_tools(scatter_hover)
#show(scatter)

In [26]:
Birds['Sps'].unique()

array(['Birds', 'mm', 'ST'], dtype=object)

In [27]:
grouped_Birds = Birds.groupby('Sps').count()
source_grouped_Birds = ColumnDataSource(grouped_Birds)
Birds_list= source_grouped_Birds.data['Sps'].tolist()
bar_Birds = figure(x_range=Birds_list)
bar_hover_Birds=HoverTool()
bar_hover.tooltips = [('No of Occourance', '@Sps')]
bar_Birds.vbar(x='Sps', top='Location', source=source_grouped_Birds,width=0.4)
bar_Birds.add_tools(bar_hover_Birds)
#show(bar) 
tab1 = Panel(child=row([plot1,bar,scatter],sizing_mode='fixed'),title="Species") 


tab2 = Panel(child=row([plot2,bar_Birds],sizing_mode='fixed'), title="Birds") 


tabs = Tabs(tabs=[tab1, tab2,tab3,tab4,tab5]) 


# show the results

#show(dash)


In [28]:

from bokeh.layouts import gridplot
dash = gridplot([[tabs]])
output_file('Dashboard.html')



In [29]:
#from bokeh.models.widgets import CheckboxGroup
# Create the checkbox selection element, available carriers is a  
# list of all airlines in the data
#carrier_selection = CheckboxGroup(labels=['Whale','Dolphin'], 
        #                          active = [0, 1])

In [30]:
show(dash)

In [ ]:
from bokeh.transform import factor_cmap
grouped = Species.groupby('Species').count()
source = ColumnDataSource(grouped)
Species_list= source.data['Species'].tolist()
bar = figure(x_range=Species_list)
bar_hover=HoverTool()
bar_hover.tooltips = [('No of Occourance', '@Type')]
bar.vbar(x='Species', top='Type', source=source,width=0.4)
bar.add_tools(bar_hover)
#show(bar) 

#from bokeh.layouts import gridplot
#output_file('Dashboard.html')
#dash = gridplot([[tabs,bar]])

# show the results

#show(dash)
